In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
from pmdarima.arima import ADFTest
from matplotlib import pyplot
import logging

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings("ignore")

In [22]:
df_train_ec = pd.read_hdf('../preprocessing/data/EC15_split.h5', key="train")
df_valid_ec = pd.read_hdf('../preprocessing/data/EC15_split.h5', key="validation")
df_test_ec = pd.read_hdf('../preprocessing/data/EC15_split.h5', key="test")

#df_train = pd.concat([df_train_ec, df_valid_ec])
df_train_ec.sort_index(inplace=True)

#get columns that end with 24 and have std in them
interesting_cols_ec = [col for col in df_train_ec.columns if col.endswith('24') and 'std' in col]

In [24]:
df_train_eg = pd.read_hdf('../preprocessing/data/GEFS_split.h5', key="train")
df_valid_eg = pd.read_hdf('../preprocessing/data/GEFS_split.h5', key="validation")
df_test_eg = pd.read_hdf('../preprocessing/data/GEFS_split.h5', key="test")

#df_train = pd.concat([df_train_eg, df_valid_eg])
df_train_eg.sort_index(inplace=True)

#get columns that end with 24 and have std in them
interesting_cols_eg = [col for col in df_train_eg.columns if col.endswith('24') and 'std' in col]

### Data

In [4]:
print("Size of the training set EC: ", df_train_ec.shape)
print("Size of the validation set EC: ", df_valid_ec.shape)
print("Size of the test set EC: ", df_test_ec.shape)

Size of the training set EC:  (3989, 30)
Size of the validation set EC:  (2393, 30)
Size of the test set EC:  (1597, 30)


In [25]:
df_train_ec.tail()

,model_0_value-FRA_WIND_EC15-24,model_0_value-FRA_WIND_EC15-48,model_0_value-FRA_WIND_EC15-72,ens_std-FRA_WIND_EC15-24,ens_std-FRA_WIND_EC15-48,ens_std-FRA_WIND_EC15-72,model_0_value-NLD_WIND_EC15-24,model_0_value-NLD_WIND_EC15-48,model_0_value-NLD_WIND_EC15-72,ens_std-NLD_WIND_EC15-24,...,model_0_value-DEU_WIND_EC15-72,ens_std-DEU_WIND_EC15-24,ens_std-DEU_WIND_EC15-48,ens_std-DEU_WIND_EC15-72,model_0_value-ESP_TEMP_EC15-24,model_0_value-ESP_TEMP_EC15-48,model_0_value-ESP_TEMP_EC15-72,ens_std-ESP_TEMP_EC15-24,ens_std-ESP_TEMP_EC15-48,ens_std-ESP_TEMP_EC15-72
2016-09-10 12:00:00,1.979452,3.127555,3.585470,0.224221,0.277322,0.404038,1.32390,2.82260,3.38893,0.437637,...,3.1021,0.276657,0.295870,0.355973,28.912284,29.702284,25.547284,0.149438,0.268809,0.859808
2016-09-11 00:00:00,1.722674,2.577522,3.236386,0.135297,0.193575,0.438772,1.29127,2.54963,3.36335,0.466491,...,2.5658,0.161485,0.202812,0.298724,22.542284,22.192284,18.607284,0.356992,0.387523,0.560340
2016-09-11 12:00:00,2.912229,3.839064,4.385529,0.211029,0.353785,0.507432,2.65377,3.25254,4.49851,0.305635,...,4.0428,0.290825,0.314667,0.405560,29.484326,24.969326,21.459326,0.240598,0.876417,0.578917
2016-09-12 00:00:00,2.436302,3.141930,2.140137,0.174454,0.340476,0.619316,2.29851,2.48460,3.33430,0.194392,...,3.3234,0.131244,0.262073,0.495850,22.824326,18.124326,16.159326,0.390801,0.436467,0.693143
2016-09-12 12:00:00,3.938436,3.366431,3.217791,0.260949,0.383843,0.772054,2.92260,3.89016,4.10063,0.300888,...,3.5675,0.249078,0.339785,0.398386,25.003928,21.058928,21.523928,0.415865,0.687078,0.568575


In [5]:
print("Size of the training set EG: ", df_train_eg.shape)
print("Size of the validation set EG: ", df_valid_eg.shape)
print("Size of the test set EG: ", df_test_eg.shape)

Size of the training set EG:  (8046, 36)
Size of the validation set EG:  (4827, 36)
Size of the test set EG:  (3220, 36)


In [31]:
df_train_eg.tail()

,model_0_value-DEU_WIND_GEFS-24,model_0_value-DEU_WIND_GEFS-48,model_0_value-DEU_WIND_GEFS-72,ens_std-DEU_WIND_GEFS-24,ens_std-DEU_WIND_GEFS-48,ens_std-DEU_WIND_GEFS-72,model_0_value-GBR_WIND_GEFS-24,model_0_value-GBR_WIND_GEFS-48,model_0_value-GBR_WIND_GEFS-72,ens_std-GBR_WIND_GEFS-24,...,model_0_value-FIN_TEMP_GEFS-72,ens_std-FIN_TEMP_GEFS-24,ens_std-FIN_TEMP_GEFS-48,ens_std-FIN_TEMP_GEFS-72,model_0_value-DEU_TEMP_GEFS-24,model_0_value-DEU_TEMP_GEFS-48,model_0_value-DEU_TEMP_GEFS-72,ens_std-DEU_TEMP_GEFS-24,ens_std-DEU_TEMP_GEFS-48,ens_std-DEU_TEMP_GEFS-72
2016-08-26 00:00:00,2.6301,3.2505,4.4342,0.092686,0.126487,0.411414,2.847656,3.462354,3.597345,0.163977,...,9.202107,0.215939,0.389660,0.818828,19.537654,20.611954,20.765854,0.141785,0.286643,0.528789
2016-08-26 06:00:00,2.4747,3.4316,5.1909,0.106431,0.215685,0.322435,2.770927,3.614951,3.114972,0.245975,...,10.652943,0.181883,0.248986,0.533443,20.459000,21.255200,18.849600,0.244659,0.302872,0.313463
2016-08-26 12:00:00,2.6693,4.8943,6.5083,0.152086,0.284184,0.601146,4.186636,5.669328,3.827965,0.470546,...,13.155798,0.196728,0.336198,0.978495,30.214200,32.014500,23.107600,0.259831,0.234422,0.400840
2016-08-26 18:00:00,3.1959,4.0543,3.3510,0.087925,0.269519,0.442990,4.548762,4.619987,3.715333,0.307824,...,12.658619,0.208302,0.315502,0.773892,25.705400,25.971800,19.208500,0.117484,0.215648,0.413266
2016-08-27 00:00:00,3.5189,3.7993,2.5488,0.072726,0.293065,0.225135,4.176639,2.991502,2.939098,0.263128,...,12.075596,0.258602,0.304878,0.692948,20.744200,19.352600,14.551900,0.183692,0.223217,0.342108


In [6]:
cols_ec = df_train_ec.columns
cols_ec

Index(['model_0_value-FRA_WIND_EC15-24', 'model_0_value-FRA_WIND_EC15-48',
       'model_0_value-FRA_WIND_EC15-72', 'ens_std-FRA_WIND_EC15-24',
       'ens_std-FRA_WIND_EC15-48', 'ens_std-FRA_WIND_EC15-72',
       'model_0_value-NLD_WIND_EC15-24', 'model_0_value-NLD_WIND_EC15-48',
       'model_0_value-NLD_WIND_EC15-72', 'ens_std-NLD_WIND_EC15-24',
       'ens_std-NLD_WIND_EC15-48', 'ens_std-NLD_WIND_EC15-72',
       'model_0_value-GBR_WIND_EC15-24', 'model_0_value-GBR_WIND_EC15-48',
       'model_0_value-GBR_WIND_EC15-72', 'ens_std-GBR_WIND_EC15-24',
       'ens_std-GBR_WIND_EC15-48', 'ens_std-GBR_WIND_EC15-72',
       'model_0_value-DEU_WIND_EC15-24', 'model_0_value-DEU_WIND_EC15-48',
       'model_0_value-DEU_WIND_EC15-72', 'ens_std-DEU_WIND_EC15-24',
       'ens_std-DEU_WIND_EC15-48', 'ens_std-DEU_WIND_EC15-72',
       'model_0_value-ESP_TEMP_EC15-24', 'model_0_value-ESP_TEMP_EC15-48',
       'model_0_value-ESP_TEMP_EC15-72', 'ens_std-ESP_TEMP_EC15-24',
       'ens_std-ESP_TEMP_EC1

In [7]:
cols_eg = df_train_eg.columns
cols_eg

Index(['model_0_value-DEU_WIND_GEFS-24', 'model_0_value-DEU_WIND_GEFS-48',
       'model_0_value-DEU_WIND_GEFS-72', 'ens_std-DEU_WIND_GEFS-24',
       'ens_std-DEU_WIND_GEFS-48', 'ens_std-DEU_WIND_GEFS-72',
       'model_0_value-GBR_WIND_GEFS-24', 'model_0_value-GBR_WIND_GEFS-48',
       'model_0_value-GBR_WIND_GEFS-72', 'ens_std-GBR_WIND_GEFS-24',
       'ens_std-GBR_WIND_GEFS-48', 'ens_std-GBR_WIND_GEFS-72',
       'model_0_value-NLD_WIND_GEFS-24', 'model_0_value-NLD_WIND_GEFS-48',
       'model_0_value-NLD_WIND_GEFS-72', 'ens_std-NLD_WIND_GEFS-24',
       'ens_std-NLD_WIND_GEFS-48', 'ens_std-NLD_WIND_GEFS-72',
       'model_0_value-FRA_WIND_GEFS-24', 'model_0_value-FRA_WIND_GEFS-48',
       'model_0_value-FRA_WIND_GEFS-72', 'ens_std-FRA_WIND_GEFS-24',
       'ens_std-FRA_WIND_GEFS-48', 'ens_std-FRA_WIND_GEFS-72',
       'model_0_value-FIN_TEMP_GEFS-24', 'model_0_value-FIN_TEMP_GEFS-48',
       'model_0_value-FIN_TEMP_GEFS-72', 'ens_std-FIN_TEMP_GEFS-24',
       'ens_std-FIN_TEMP_GEF

### Test for stationarity

In [ ]:
for col in uncertainty_columns:
    try:
        if ADFTest(alpha=0.05).should_diff(dataTrain[col])[1] == True:
            print("Column {} is not stationary".format(col))
    except BaseException as exception:
        print("Column {} has error: {}".format(col, exception))

### Trying to predict out of sample uncertainty

In [ ]:
uncertainty_columns = ['ens_std-DEU_TEMP_EC15-138','mae-DEU_TEMP_EC15-30']
for col in uncertainty_columns:
    model = ARIMA(dataTrain[col], order=(1,0,0))
    model_fit = model.fit()
    model_pred = model_fit.predict(start=np.size(dataTrain,0), end=np.size(dataTrain,0)+np.size(dataVal,0), typ='levels')[1:]
    model_pred.index = dataVal.index

    # plot results for the training set
    pyplot.plot(dataTrain[col], label='actual')
    pyplot.plot(model_fit.fittedvalues, label='predicted')
    pyplot.legend()
    #save the plots
    pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'training', col + '.png'))
    pyplot.show()

    # plot results for the validation set
    pyplot.plot(dataVal[col], label='actual')
    pyplot.plot(model_pred, label='predicted')
    pyplot.legend()
    #save the plots
    pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'validation', col + '.png'))
    pyplot.show()

##### In the validation set, the model converges to the long term mean as expected

### Simply predicting one step ahead

In [ ]:
uncertainty_columns = ['ens_std-DEU_TEMP_EC15-138','mae-DEU_TEMP_EC15-30']
data = dataTrain.append(dataVal)
nTrain = np.size(dataTrain,0)

for col in uncertainty_columns:

    valPred = []
    for t in range(len(dataVal)):
        model = ARIMA(data[col][:nTrain+t], order=(1,1,0))
        model_fit = model.fit()
        yhat = model_fit.forecast()
        valPred.append(float(yhat))
    dataVal[col + '_pred'] = valPred

    # plot results for the validation set
    pyplot.plot(dataVal[col], label='actual')
    pyplot.plot(dataVal[col + '_pred'], label='predicted')
    pyplot.legend()
    #save the plots
    pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'validation', col + '.png'))
    pyplot.show()
    

In [ ]:
# calclulate the mean squared error for the validation set
for col in uncertainty_columns:
    mse = np.mean((dataVal[col].values - dataVal[col + '_pred'].values)**2)
    print("The mean squared error for {} is: {}".format(col, mse))

### Predicting one step ahead for all columns

In [ ]:
def univariate_uncertainty_prediction(dataTrain, dataVal):
    std_columns = [col for col in dataTrain.columns if 'std' in col]
    mae_columns = [col for col in dataTrain.columns if 'mae' in col]
    uncertainty_columns = std_columns + mae_columns

    data[uncertainty_columns] = dataTrain[uncertainty_columns].append(dataVal[uncertainty_columns])
    nTrain = np.size(dataTrain,0)

    for col in uncertainty_columns:
        valPred = []
        for t in range(len(dataVal)):
            model = ARIMA(data[col][:nTrain+t], order=(1,0,0))
            model_fit = model.fit()
            yhat = model_fit.forecast()
            valPred.append(float(yhat))
        dataVal[col + '_pred'] = valPred
    
    return dataVal

In [ ]:
predVal = univariate_uncertainty_prediction(dataTrain, dataVal)

In [ ]:
std_columns = [col for col in dataTrain.columns if 'std' in col]
mae_columns = [col for col in dataTrain.columns if 'mae' in col]
uncertainty_columns = std_columns + mae_columns

In [ ]:
# calclulate the mean squared error for the validation set
mse_std = []
for col in std_columns:
    mse = np.mean((dataVal[col].values - dataVal[col + '_pred'].values)**2)
    mse_std.append(mse)

#plot the results
pyplot.plot(mse_std, label='mse_std')
pyplot.legend()
pyplot.show()

In [ ]:
mse_mae = []
for col in mae_columns:
    mse = np.mean((dataVal[col].values - dataVal[col + '_pred'].values)**2)
    mse_mae.append(mse)

#plot the results
pyplot.plot(mse_mae, label='mse_mae')
pyplot.legend()
pyplot.show()

### For specific ValueDate, see ensemble uncertainty vs predicted uncertainty for different ForecastHorizon

#### For Uncertainty measured as STD

In [ ]:
predVal_std = dataVal[[col for col in dataVal.columns if 'std' in col and '_pred' in col]]
dataVal_std = dataVal[[col for col in dataVal.columns if 'std' in col and '_pred' not in col]]
std_pred = [predVal_std.iloc[-1,0]]
std_true = [dataVal_std.iloc[-1,0]]
for iter in range(1,int(len(std_columns)/2)):
    row = np.size(dataVal,0) - iter - 1
    col = 2 * iter
    std_pred.append(predVal_std.iloc[row,col])
    std_true.append(dataVal_std.iloc[row,col])

In [ ]:
# plot std_pred vs std_true
x_axis = np.arange(12, 372, 12)
pyplot.plot(x_axis, std_pred[1:], label='std_pred')
pyplot.plot(x_axis, std_true[1:], label='std_true')
pyplot.legend()
#save the plots
pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'validation', 'std_pred_vs_std_true.png'))
pyplot.show()

#### For Uncertainty measured as MAE

In [ ]:
predVal_mae = dataVal[[col for col in dataVal.columns if 'mae' in col and '_pred' in col]]
dataVal_mae = dataVal[[col for col in dataVal.columns if 'mae' in col and '_pred' not in col]]
mae_pred = [predVal_mae.iloc[-1,0]]
mae_true = [dataVal_mae.iloc[-1,0]]
for iter in range(1,int(len(std_columns)/2)):
    row = np.size(dataVal,0) - iter - 1
    col = 2 * iter
    mae_pred.append(predVal_mae.iloc[row,col])
    mae_true.append(dataVal_mae.iloc[row,col])

In [ ]:
# plot mae_pred vs mae_true
x_axis = np.arange(12, 372, 12)
pyplot.plot(x_axis, mae_pred[1:], label='predicted')
pyplot.plot(x_axis,mae_true[1:], label='actual')
pyplot.legend()
#save the plots
pyplot.savefig(os.path.join(os.path.dirname(os.getcwd()), 'models', 'plots', 'univariate', 'validation', 'mae_pred_vs_mae_true.png'))
pyplot.show()

### ARIMA EVALUATION

In [18]:
def train_predict_save_arima(train, valid, test, cols, order, window, save_path):
    
    data = train.append(valid)
    data = data.append(test)
    nTrain = np.size(train,0)
    nValid = np.size(valid,0)
    nTest = np.size(test,0)

    for col in cols:

        print("Validation Training and predicting for {}".format(col))
        y_valid_hat = []
        for t in range(nValid):
            model = sm.tsa.arima.ARIMA(data[col][nTrain+t-window:nTrain+t], order=(1,1,0))
            model_fit = model.fit()
            y_hat = model_fit.forecast(steps=2)[1]
            y_valid_hat.append(float(y_hat))
        valid[col + '_pred'] = y_valid_hat

        print("Test Training and predicting for {}".format(col))
        y_test_hat = []
        for t in range(nTest):
            model = sm.tsa.arima.ARIMA(data[col][nTrain+t-window:nTrain+nValid+t], order=(1,1,0))
            model_fit = model.fit()

            y_hat = model_fit.forecast(steps=2)[1]
            y_test_hat.append(float(y_hat))

            # save the model for the last few days fitted until this point with index of the last observation
            if t >= nTest-11:
                last_row = data[col].index[nTrain+nValid+t]
                model_fit.save(os.path.join(save_path, 'arima_model_' + col + '_' + str(last_row) + '.pkl'))
                
        test[col + '_pred'] = y_test_hat

    valid.to_csv(os.path.join(save_path, 'arima_valid.csv'))
    test.to_csv(os.path.join(save_path, 'arima_test.csv'))
    
    return valid, test

In [39]:
col = interesting_cols_ec[0]
window = 12
order = (1,1,0)
model = sm.tsa.arima.ARIMA(df_test_ec[col][0:15], order=(1,1,0))
model_fit = model.fit()
y_hat = model_fit.forecast(steps=2)[1]

In [40]:
y_hat

0.10578440178854236

In [ ]:
def evaluate_arima(valid, test, cols):
    from sklearn.metrics import r2_score
    #calculate r2 score for the validation set for each column
    r2_valid = []
    for col in cols:
        r2 = r2_score(valid[col].values, valid[col + '_pred'].values)
        r2_valid.append(r2)
        #print r2 of the column
        print("Validation r2 score for {} is {}".format(col, r2))
    
    #calculate r2 score for the test set
    r2_test = []
    for col in cols:
        r2 = r2_score(test[col].values, test[col + '_pred'].values)
        r2_test.append(r2)
        #print r2 of the column
        print("Test r2 score for {} is {}".format(col, r2))

    return r2_valid, r2_test

In [19]:
save_path = os.path.join(os.path.dirname(os.getcwd()), 'models', 'model_save','arima')
#look at last 5 years for training
window = 360 * 5

In [20]:
valid_ec, test_ec = train_predict_save_arima(df_train_ec, df_valid_ec, df_train_ec, interesting_cols_ec, (1,1,0), window, save_path)

Validation Training and predicting for model_0_value-FRA_WIND_EC15-24
Test Training and predicting for model_0_value-FRA_WIND_EC15-24
Validation Training and predicting for ens_std-FRA_WIND_EC15-24


KeyboardInterrupt: 